In [18]:
import os, pandas as pd, math

os.chdir('/Users/[editted]/Desktop/compSoc/datasets/MA_cbg_human_mobility_2020')
ma2_df = pd.read_csv('MA_cbg2cbg_2020_02.csv')
ma4_df = pd.read_csv('MA_cbg2cbg_2020_04.csv')

ma_df = pd.concat([ma2_df, ma4_df])
ma_df = ma_df.astype({"year_month": "int"})

visitor_df = ma_df.groupby(["poi_cbg", 
                            "year_month"]).agg({"visitor_count": "sum"}).reset_index() #how many people traveled from a location

ym_df = ma_df.groupby(["year_month"]).agg({"visitor_count": "sum"}).reset_index() 
ym_mean_df = ma_df.groupby(["year_month"]).agg({"visitor_count": "mean"}).reset_index() 
visitor_df2 = visitor_df[visitor_df["year_month"]==2020_2]
visitor_df4 = visitor_df[visitor_df["year_month"]==2020_4]
pd.DataFrame.var(visitor_df2) #var 3.729 * 10^6
print(math.sqrt(3.729 * 10**6)) #stdev 1931.061
pd.DataFrame.var(visitor_df4) #var 4.205 * 10^5
print(math.sqrt(4.205 * 10**5)) #stdev 648.460
print("Max Visitor Count Feb: ", pd.DataFrame.max(visitor_df2["visitor_count"]), 
      "\nMax Visitor Count Apr: ", pd.DataFrame.max(visitor_df4["visitor_count"]))

ym_df
ym_mean_df



1931.0618840420418
648.459713474939
Max Visitor Count Feb:  69792 
Max Visitor Count Apr:  16745


,year_month,visitor_count
0,20202,6.931361
1,20204,7.260037


Overall, visitor counts decreased from 6.5 million to 2.8 million, a stark decrease due to Covid-19. The maximum number of visitors from one place of interest in February was 69,792, while in April, it was only 16,745. Interestingly, the average number of visitors to each POI increased from Feb to April—it may have been that high-traffic POIs saw large decreases in visitor counts, but low-traffic POIs saw increases to offset this, due to people wanting to go out to smaller places (lower risk of Covid-19). Finally, the standard deviation of visitor counts decreased drastically from Feb to April, indicating that not only were POIs seeing fewer visitors, the number of visitors from each became much more clustered together.

In [33]:
import os, pandas as pd, networkx as nx

os.chdir('/Users/[editted]/Desktop/compSoc/datasets/MA_cbg_human_mobility_2020')

ma2_df = pd.read_csv('MA_cbg2cbg_2020_02.csv')
ma4_df = pd.read_csv('MA_cbg2cbg_2020_04.csv')

G2 = nx.DiGraph()

for i in range(len(ma2_df)):
    origin_cbg = ma2_df.loc[i, 'poi_cbg']
    destination_cbg = ma2_df.loc[i, 'visitor_census_tract']
    if str(origin_cbg)[:2] == '25' and str(destination_cbg)[:2] == '25':
        total_residents_in_origin = ma2_df.loc[i, 'raw_device_counts']
        num_visitor = ma2_df.loc[i, 'visitor_count']
        pct_visitor = num_visitor / total_residents_in_origin
        G2.add_edge(origin_cbg, destination_cbg, weight=pct_visitor)
    
G4 = nx.DiGraph()

for i in range(len(ma4_df)):
    origin_cbg = ma4_df.loc[i, 'poi_cbg']
    destination_cbg = ma4_df.loc[i, 'visitor_census_tract']
    if str(origin_cbg)[:2] == '25' and str(destination_cbg)[:2] == '25':
        total_residents_in_origin = ma4_df.loc[i, 'raw_device_counts']
        num_visitor = ma4_df.loc[i, 'visitor_count']
        pct_visitor = num_visitor / total_residents_in_origin
        G4.add_edge(origin_cbg, destination_cbg, weight=pct_visitor)
    
print(G2, "\n", G4)

close2 = nx.closeness_centrality(G2)
csum2 = 0
cnum2 = 0
for i in close2.values():
    csum2 += i
    cnum2 += 1
cmean2 = csum2/cnum2

close4 = nx.closeness_centrality(G4)
csum4 = 0
cnum4 = 0
for i in close4.values():
    csum4 += i
    cnum4 += 1
cmean4 = csum4/cnum4

print("Feb: ", cmean2, "Apr: ", cmean4)

bet2 = nx.betweenness_centrality(G2)
bsum2 = 0
bnum2 = 0
for i in close2.values():
    bsum2 += i
    bnum2 += 1
bmean2 = bsum2/bnum2

bet4 = nx.betweenness_centrality(G4)
bsum4 = 0
bnum4 = 0
for i in bet4.values():
    bsum4 += i
    bnum4 += 1
bmean4 = bsum4/bnum4

print("Feb: ", bmean2, "Apr: ", bmean4)

print("Feb: ", nx.average_clustering(G2), "Apr: ", nx.average_clustering(G4))

DiGraph with 4984 nodes and 745666 edges 
 DiGraph with 4985 nodes and 339844 edges
Feb:  0.2534210545393452 Apr:  0.21561300394680086


Closeness centrality decreased on average from 0.4349 to 0.3667 Feb-Apr; on average, nodes were farther apart (takes longer to send information from one node through the others) in April, indicating that communities in this network were less-connected. Betweeness centrality decreased very drastically, becoming nearly zero in April. A betweeness centrality of zero would indicate that each node is not between any other nodes; this shows that travelers did not travel through hubs as they did prior to Covid. Average clustering coefficient for G also decreased slightly, from 0.2534 to 0.2156. This indicates that CBGs are more separate than before Covid, which aligns with the other metrics calculated here.